# Tugas Besar - Regresi Linear

Anggota Kelompok:

- Quin Derbi Kusuma (1305210071)

- Muhammad Ali Akbar Al-Qahri (1305210077)

- Dominica Febryanti (1305213010)

# Import Library

Pada tugas kali ini, digunakan beberapa library untuk menganalisis data menggunakan Python. Library yang digunakan yaitu :

- Pandas, library yang menyediakan struktur data dan alat untuk manipulasi data.

- Numpy, library yang digunakan untuk melakukan operasi matematika dan ilmiah pada array numerik.

- Fungsi f dari library spicy.stats, yang digunakan untuk menghitung distribusi F dalam statistika.

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import f

# Dataset

### Read Data Frame

Data yang digunakan merupakan dataset yang memuat 29 baris dan 149 baris data berisi informasi tentang berbagai aspek ekonomi, sosial, dan demografi dari berbagai negara. 

In [2]:
df = pd.read_csv('factbook.csv')
df.head()

,Country,Area,Birth rate,Current account balance,Death rate,Electricity consumption,Electricity production,Exports,GDP,GDP per capita,...,Life expectancy at birth,Military expenditures,Natural gas consumption,Oil consumption,Population,Public debt,Railways,Reserves of foreign exchange & gold,Total fertility rate,Unemployment rate
0,Albania,28748,15.08,504000000,5.12,6760000000,5680000000,552400000,17460000000,4900,...,77.24,1.49,3.000000e+07,7500,3563112,NaN,447.0,1.206000e+09,2.04,14.8
1,Algeria,2381740,17.13,11900000000,4.60,23610000000,25760000000,32160000000,212300000000,6600,...,73.00,3.20,2.232000e+10,209000,32531853,37.4,3973.0,4.355000e+10,1.92,25.4
2,Angola,1246700,44.64,37880000,25.90,1587000000,1707000000,12760000000,23170000000,2100,...,36.61,10.60,5.300000e+08,31000,11190786,NaN,2761.0,8.000000e+08,6.27,NaN
3,Argentina,2766890,16.90,5473000000,7.56,81650000000,81390000000,33780000000,483500000000,12400,...,75.91,1.30,3.110000e+10,486000,39537943,118.0,34091.0,1.947000e+10,2.19,14.8
4,Armenia,29800,11.76,240400000,8.16,5797000000,6492000000,850000000,13650000000,4600,...,71.55,6.50,1.400000e+09,5700,2982904,NaN,845.0,5.550000e+08,1.32,30.0


Untuk membuat regresi linier dengan metode regresi bartahap, digunakan GDP sebagai variabel dependen (Y), dan digunakan Export, Import, Industrial production growth rate, Investment, dan Unemployment rate sebagai variabel independen (X).

In [3]:
X = ['Exports','Imports','Industrial production growth rate','Investment','Unemployment rate']
Y = ['GDP']

### Select Data

In [4]:
data = df[X+Y]
data = data.dropna()
data.head()

,Exports,Imports,Industrial production growth rate,Investment,Unemployment rate,GDP
0,552400000,2076000000,3.1,18.4,14.8,17460000000
1,32160000000,15250000000,6.0,26.2,25.4,212300000000
3,33780000000,22060000000,12.0,18.3,14.8,483500000000
4,850000000,1300000000,15.0,19.8,30.0,13650000000
5,86890000000,98100000000,1.9,25.3,5.1,611700000000


# Tahap Persiapan

## Simplify Data

Dilakukan penyederhaan pada data dengan nilai yang sangat besar yaitu GDP, Export, dan Import. Hal ini dilakukan untuk mencegah terjadinya masalah presisi numerik pada Python. Dengan dilakukan penyederhanaan dapat dipastikan bahwa data dapat diproses dengan benar tanpa mengorbankan akurasi hasil analisis.

In [5]:
# simplify the data
data['GDP'] = data['GDP'] / 1000000
data['Exports'] = data['Exports'] / 1000
data['Imports'] = data['Imports'] / 1000
# Result
data.head()

,Exports,Imports,Industrial production growth rate,Investment,Unemployment rate,GDP
0,552400.0,2076000.0,3.1,18.4,14.8,17460.0
1,32160000.0,15250000.0,6.0,26.2,25.4,212300.0
3,33780000.0,22060000.0,12.0,18.3,14.8,483500.0
4,850000.0,1300000.0,15.0,19.8,30.0,13650.0
5,86890000.0,98100000.0,1.9,25.3,5.1,611700.0


## Deklarasi Fungsi Dan Konstanta

Pendeklarasian fungsi yang akan digunakan yaitu:
Jumlah Kuadrat Total (JKT) yang merupakan ukuran total variasi dalam variabel dependen (Y).

$JKT = J_{yy} = n\sum_{i=1}^{n} (y_i - \bar{Y}) = n\sum_{i=1}^{n} y_i^2 - \frac{(n\sum_{i=1}^{n} y_i)}{n}$ 

Jumlah Kuadrat Regresi(JKR) yang merupakan ukuran variasi atau variasibilitas dalam variabel dependen (Y) yang dapat dijelaskan oleh model regresi.

$ JKR = \sum_{i=1}^{k} b_jg_j - \frac{(\sum_{i=1}^{n} y_i)^2}{n}$ 

Rumus untuk mendapatkan nilai $\beta$ untuk setiap peubah yaitu:

$\beta = (X^TX)^{-1}X^TY$

dimana;

$Y = 
\begin{pmatrix}
y_{1} \\
y_{2}\\
\vdots \\
y_{n} 
\end{pmatrix}
\qquad \qquad 
X=
\begin{pmatrix}
1&x_{1,1}&x_{1,2}&\dots&x_{1,k}\\
1&x_{2,1}&x_{2,2}&\dots&x_{2,k}\\
\vdots &\vdots &\vdots&\ddots&\vdots\\
1&x_{n,1}&x_{n,2}&\dots&x_{n,k}
\end{pmatrix}  
\qquad \qquad 
\beta =
\begin{pmatrix}
\beta_{1} \\
\beta_{2}\\
\vdots \\
\beta_{n} 
\end{pmatrix} 
$

In [6]:
def jkt(y_arr):
    return np.sum(y_arr**2)-np.sum(y_arr)**2/len(y_arr)

def jkr(x_arr,y_arr):
    beta = np.linalg.inv(x_arr.transpose().dot(x_arr)).dot(x_arr.transpose()).dot(y_arr)
    g = x_arr.transpose().dot(y_arr)
    return (sum(beta*g) - np.sum(y_arr)**2/len(y_arr))[0]

def b(x_arr,y_arr):
    return np.linalg.inv(x_arr.T @ x_arr) @ x_arr.T @ y_arr

 N mewakili jumlah baris atau entri dalam data.

JKT sebagai variabel tempat penyimpanan hasil dari pemanggilan fungsi jkt dengan menggunakan nilai dai GDP sebagai parameter fungsi.

In [7]:
N = len(data)
JKT = jkt(data['GDP'].values)
print(f'N = {N}')
print(f'JKT = {JKT}')

N = 116
JKT = 217544221905839.8


In [8]:
remaining = X.copy()
selected = []

# Tahap Regresi

In [9]:
print('# STEP 1')
print()
print('PEMILIHAN MAJU')
R_Bi = dict()
for i in remaining:
    
    y_arr = np.array(data[Y])
    
    b0 = np.ones((N,1))
    x_arr = np.array(data[i])
    x_arr = np.concatenate((b0,x_arr.reshape(-1,1)),axis=1)
    
    r = jkr(x_arr,y_arr)
    R_Bi[i] = r
    
for k,v in R_Bi.items():
    print(f'R({k}) = {v}')
print()

max_r = max(R_Bi, key=R_Bi.get)

print(f'max_r : {max_r}')

k = len(selected)+1

s2 = ((JKT-R_Bi[max_r])/(N-k-1))
print(f's^2 = {s2}')

f_hitung = R_Bi[max_r]/s2
f_tabel = f.ppf(0.95,1,N-k-1)
print(f'f_hitung = {f_hitung}')
print(f'f_tabel = {f_tabel}')
if f_hitung > f_tabel:
    print(f'f_hitung > f_tabel, maka {max_r} masuk model')
    selected.append(max_r)
    remaining.remove(max_r)
else:
    print(f'f_hitung < f_tabel, maka {max_r} tidak masuk model')
    remaining.remove(max_r)

print()
print(f'X = {selected}')

# STEP 1

PEMILIHAN MAJU
R(Exports) = 128742660966327.2
R(Imports) = 172563979583237.8
R(Industrial production growth rate) = 243499249762.26172
R(Investment) = 695704935452.4062
R(Unemployment rate) = 5233620874369.273

max_r : Imports
s^2 = 394563529145.6316
f_hitung = 437.35410608501843
f_tabel = 3.924330484639666
f_hitung > f_tabel, maka Imports masuk model

X = ['Imports']


## langkah 1

Pada langka ini merupakan implementasi dari langka pemilihan fitur dengan menggunakan metode korelasi JKR. Pada setiap iterasi, fitur dengan korelasi JKR tertinggi dipilih untuk dimasukkkan kedalam model, jika mememuji kondisi f-hitung > f-tabel. Setelah pemilihan fitur selesai, hasilnya ditampilkan dalam bentuk daftar fitur yang sudah terpilih

Membuat persamaan regresi sederhana GDP terhadap 5 peubah bebas. Jumlah kuadrat regresi dari masing-masing model adalah :

$ R(Export) = 128742660966327.2$
$ R(Import) = 172563979583237.8$
$ R(IndustrialProductionGrowRate) = 243499249762.26172$
$ R(Invesment) = 695704935452.4062$
$ R(UnemploymentRate) = 5233620874369.273$

Peubah Imports mempunyai nilai R terbesar, dikarenakan

$f_{hitung} = \frac{R(Import)}{S^2} = \frac{R(Imports)}{(\frac{JKT-R(Imports)}{N-k-1})}= 437.35410608501843> f_{tabel_{0,95(1,114)}} = 3.924330484639666 $

karena $f_{hitung}$ lebih besar dari $f_{tabel}$ maka peubah $import$ masuk ke dalam model.

In [10]:
step = 2
while(True):
    print()
    print(f'# STEP {step}')
    print()
    print('PEMILIHAN MAJU')
    
    R_Bi = dict()
    R_Bi_join = dict()
    for i in remaining:
        
        y_arr = np.array(data[Y])
        
        b0 = np.ones((N,1))
        x_arr_selected = np.concatenate((b0,np.array(data[selected])),axis=1)
        x_arr = np.concatenate((x_arr_selected,np.array(data[i]).reshape(-1,1)),axis=1)
        
        r_selected = jkr(x_arr_selected,y_arr)
        r = jkr(x_arr,y_arr)
        
        R_Bi[i] = r - r_selected
        R_Bi_join[i] = r
        
    for k,v in R_Bi.items():
        print(f'R({k}|{selected}) = {v}')
    print()

    max_r = max(R_Bi, key=R_Bi.get)

    print(f'max_r : {max_r}')

    k = len(selected)+1
    print(f'k = {k}')

    s2 = ((JKT-R_Bi_join[max_r])/(N-k-1))
    print(f's^2 = {s2}')
    
    f_hitung = R_Bi[max_r]/s2
    f_tabel = f.ppf(0.95,1,N-k-1)
    print(f'f_hitung = {f_hitung}')
    print(f'f_tabel = {f_tabel}')
    if f_hitung > f_tabel:
        print(f'f_hitung > f_tabel, maka {max_r} masuk model')
        selected.append(max_r)
        remaining.remove(max_r)
    else:
        print(f'f_hitung < f_tabel, maka {max_r} tidak masuk model')
        remaining = []
        print()
        break
    
    print()
    print('PENYISIHAN MUNDUR')
    
    checker = True
    while(checker):
        R_Bi = dict()
        R_Bi_join = dict()
        dict_selected = dict()
        for i in selected[0:-1]:
            
            x_selected_temp = selected.copy()
            x_selected_temp.remove(i)
            dict_selected[i] = x_selected_temp
            
            y_arr = np.array(data[Y])
            
            b0 = np.ones((N,1))
            x_arr_selected_temp = np.concatenate((b0,np.array(data[x_selected_temp])),axis=1)
            x_arr = np.concatenate((x_arr_selected_temp,np.array(data[i]).reshape(-1,1)),axis=1)
            
            r = jkr(x_arr,y_arr)
            r_selected_temp = jkr(x_arr_selected_temp,y_arr)
            
            R_Bi[i] = r - r_selected_temp
            R_Bi_join[i] = r
            
        for k,v in R_Bi.items():
            print(f'R({k}|{dict_selected[k]}) = {v}')
            
        min_r = min(R_Bi, key=R_Bi.get)

        print(f'min_r : {min_r}')

        k = len(dict_selected[min_r])+1
        print(f'k = {k}')

        f_hitung = R_Bi[min_r]/((JKT-R_Bi_join[min_r])/(N-k-1))
        f_tabel = f.ppf(0.95,1,N-k-1)
        print(f'f_hitung = {f_hitung}')
        print(f'f_tabel = {f_tabel}')
        if f_hitung < f_tabel:
            print(f'f_hitung < f_tabel, maka {min_r} keluar dari model')
            selected.remove(min_r)
            remaining.append(min_r)
        else:
            print(f'f_hitung > f_tabel, maka {min_r} tetap dalam model')
            checker = False
            
    print()
    print(f'X = {selected}')
    print()
    if len(remaining) == 0:
        break
    step+=1
    
print(f'X = {selected}')


# STEP 2

PEMILIHAN MAJU
R(Exports|['Imports']) = 4055901768883.9375
R(Industrial production growth rate|['Imports']) = 2155686544400.6875
R(Investment|['Imports']) = 1100268262274.75
R(Unemployment rate|['Imports']) = 300721957470.625

max_r : Exports
k = 2
s^2 = 362162305785.1156
f_hitung = 11.199127308656069
f_tabel = 3.9250756165304113
f_hitung > f_tabel, maka Exports masuk model

PENYISIHAN MUNDUR
R(Imports|['Exports']) = 47877220385794.61
min_r : Imports
k = 2
f_hitung = 132.19824266913636
f_tabel = 3.9250756165304113
f_hitung > f_tabel, maka Imports tetap dalam model

X = ['Imports', 'Exports']


# STEP 3

PEMILIHAN MAJU
R(Industrial production growth rate|['Imports', 'Exports']) = 2049409777337.1875
R(Investment|['Imports', 'Exports']) = 1327706597232.5625
R(Unemployment rate|['Imports', 'Exports']) = 94454291606.4375

max_r : Industrial production growth rate
k = 3
s^2 = 347097596217.6864
f_hitung = 5.904419390020425
f_tabel = 3.9258342687862307
f_hitung > f_tabel, maka Indus

Pada langkah ke-2, dilakukan pemilihan maju dengan menambahkan fitur ke dalam model yang memiliki korelasi JKR terbesar dengan fitur-fitur yang sudah terpilih sebelumnya. Dalam kasus ini, fitur "Exports" memiliki korelasi JKR terbesar dengan fitur "Imports". Setelah ditambahkan ke dalam model, dilakukan perhitungan f-hitung dan f-tabel untuk menguji signifikansi fitur tersebut. Hasilnya, f-hitung (11.199) lebih besar daripada f-tabel (3.925), sehingga fitur "Exports" dinyatakan signifikan dan dimasukkan ke dalam model.

Selanjutnya, dilakukan penyisihan mundur untuk memeriksa fitur-fitur yang sudah terpilih. Fitur "Imports" diuji dengan menghapusnya dari model dan menghitung ulang korelasi JKR dengan fitur yang masih tersisa. Dalam kasus ini, f-hitung (132.198) juga lebih besar daripada f-tabel (3.925), sehingga fitur "Imports" tetap dalam model.Sebagai hasilnya, pada langkah ke-2, fitur "Imports" dan "Exports" telah dipilih untuk dimasukkan ke dalam model. Summary dari pemilihan fitur ini adalah:

Fitur terpilih (X):

Imports
Exports
Fitur-fitur ini dipilih karena memiliki korelasi JKR yang signifikan dengan variabel target dan tidak dihapus saat proses penyisihan mundur. Dengan menggunakan fitur-fitur ini, diharapkan model dapat memiliki kemampuan yang lebih baik dalam memprediksi atau menjelaskan variabel target yang sedang diamati.


Setelah peubah Imports masuke ke dalam model, maka 4 persamaan regresi yang mengandung peubah Imports, yaitu (Imports, Exports), (Imports, Industrial Production Growth Rate), (Imports, Investment), dan (Imports, Unemployment Rate) dicocokkan pada langkah ini. Jumlah Kuadrat Regresi untuk keempat persamaan regresi tersebut adalah:

$ R(Exports|Imports) = 4055901768883.9375 $
$ R(Industrial production growth rate|Imports) = 2155686544400.6875 $
$ R(Investment|Imports) = 1100268262274.75 $
$ R(Unemployment rate|Imports) = 300721957470.625 $

Peubah Exports memberikan jumlah kuadrat regresi terbesar jika Imports ada di dalam model.
Regresi antara Imports dan Exports menghasilkan  S^2 yang baru, yaitu S^2= 362162305785.1156. Sehingga diperoleh:

$ f\ \ hitung=\ \frac{R(Exports|Imports)}{S^2}=\frac{4055901768883.9375}{362162305785.1156}\ =\ 11.199127308656069 > {f\ tabel}_{0.95(1,113)}\ =\ 3.9250756165304113 $

Maka peubah Exports masuk dalam model bersama Imports.
Setelah itu, perlu diuji apakah peubah Imports masih berarti dalam model jika Exports ada di dalam model. Diperoleh Jumlah Kuadrat Regresi R(Imports|Exports) = 47877220385794.61, sehingga

$ f\ \ hitung=\ \frac{R(Imports|Exports)}{S^2}=\frac{47877220385794.61}{362162305785.1156}\ =\ 131.02834671631214 > {f\ tabel}_{0.95(1,113)}\ =\ 3.9250756165304113 $

karena $f_{hitung}$ lebih besar dari $f_{tabel}$ maka peubah $Import$ tetap dalam model bersama $Exports$.


## langkah 3

Pada langkah ke-3, dilakukan pemilihan maju dengan menambahkan fitur ke dalam model yang memiliki korelasi JKR terbesar dengan fitur-fitur yang sudah terpilih sebelumnya. Dalam kasus ini, fitur "Industrial production growth rate" memiliki korelasi JKR terbesar dengan fitur "Imports" dan "Exports". Setelah ditambahkan ke dalam model, dilakukan perhitungan f-hitung dan f-tabel untuk menguji signifikansi fitur tersebut. Hasilnya, f-hitung (5.904) lebih besar daripada f-tabel (3.925), sehingga fitur "Industrial production growth rate" dinyatakan signifikan dan dimasukkan ke dalam model.

Selanjutnya, dilakukan penyisihan mundur untuk memeriksa fitur-fitur yang sudah terpilih. Fitur "Imports" dan "Exports" diuji dengan menghapusnya satu per satu dari model dan menghitung ulang korelasi JKR dengan fitur yang masih tersisa. Dalam kasus ini, f-hitung untuk kedua fitur tersebut (11.379 untuk "Exports" dan 0.0 untuk "Imports") juga lebih besar daripada f-tabel (3.925), sehingga kedua fitur tersebut tetap dalam model.

Sebagai hasilnya, pada langkah ke-3, fitur "Imports", "Exports", dan "Industrial production growth rate" telah dipilih untuk dimasukkan ke dalam model. Summary dari pemilihan fitur ini adalah:

Fitur terpilih (X):

Imports
Exports
Industrial production growth rate
Fitur-fitur ini dipilih karena memiliki korelasi JKR yang signifikan dengan variabel target dan tidak dihapus saat proses penyisihan mundur. Dengan menggunakan fitur-fitur ini, diharapkan model dapat memiliki kemampuan yang lebih baik dalam memprediksi atau menjelaskan variabel target yang sedang diamati.

Setelah Imports dan Exports masuk ke dalam model, dihitung Jumlah Kuadrat Regresi untuk model regresi dengan memasukkan peubah Industrial Production Growth Rate, Investment, dan UnemploymentRate

$ R(Industrial production growth rate|Imports, Exports) = 2049409777337.1875 $
$ R(Investment|Imports, Exports) = 1327706597232.5625 $
$ R(Unemployment rate|Imports, Exports) = 94454291606.4375 $

Peubah Industrial production growth rate memberikan jumlah kuadrat regresi terbesar jika Imports dan Export ada di dalam model. Regresi antara Industrial production growth rate dan Import,Exports menghasilkan  $S^2$ yang baru, yaitu $S^2\ =\ 347097596217.6864$. Sehingga diperoleh

$ f\ \ hitung=\ \frac{R(Industrial\ production\ growth\ rate\ |Imports,\ Import)}{S^2} =\ 5.904419390020425  > {f\ tabel}_{0.95(1,112)}\ =3.9258342687862307 $

karena $f_{hitung}$ lebih besar dari $f_{tabel}$ maka peubah Industrial production growth rate masuk dalam model. 

Setelah itu, perlu diuji apakah peubah Imports dan export masih berarti dalam model jika Industrial production growth rate  ada di dalam model. Diperoleh Jumlah Kuadrat Regresi 

$ R(Imports|Exports, Industrial production growth rate) = 48101701029920.234 $ 
$ R(Exports|Imports, Industrial production growth rate) = 3949625001820.4375 $ 

Dipilih peubah dengan R terkecil yaitu Export  untuk diuji $f hitungnya$

$ f\ \ hitung=\ \frac{R(Exports |Import,\ Industrial production growth rate)}{S^2} =\ 11.379004190347038  > {f\ tabel}_{0.95(1,112)}\ =3.9258342687862307 $

karena $f_{hitung}$ lebih besar dari $f_{tabel}$ maka peubah $Import dan Exports$ tetap dalam model bersama $Industrial\ production\ growth\ rate$ .



## Langkah 4

Pada langkah ke-4, dilakukan pemilihan maju dengan menambahkan fitur "Investment" ke dalam model yang sudah terpilih sebelumnya, yaitu fitur "Imports", "Exports", dan "Industrial production growth rate". Setelah ditambahkan, dilakukan perhitungan korelasi JKR dan perhitungan f-hitung dan f-tabel untuk menguji signifikansi fitur "Investment" dalam model.

Hasilnya, korelasi JKR antara "Investment" dengan variabel target tidak signifikan dibandingkan dengan variabel target dan fitur-fitur yang sudah ada dalam model. Dalam hal ini, f-hitung (1.876) lebih kecil daripada f-tabel (3.927), sehingga fitur "Investment" tidak signifikan dan tidak dimasukkan ke dalam model.

Sebagai hasilnya, pada langkah ke-4, fitur yang tetap dalam model adalah "Imports", "Exports", dan "Industrial production growth rate". Summary dari pemilihan fitur ini adalah:

Fitur terpilih (X):

Imports
Exports
Industrial production growth rate


Fitur-fitur ini dipilih karena memiliki korelasi JKR yang signifikan dengan variabel target dan tidak dihapus saat proses penyisihan mundur. Dengan menggunakan fitur-fitur ini, diharapkan model dapat memiliki kemampuan yang lebih baik dalam memprediksi atau menjelaskan variabel target yang sedang diamati. Fitur "Investment" tidak dimasukkan ke dalam model karena korelasinya yang tidak signifikan.

Setelah Imports, Exports, dan Industrial Production Growth Rate masuk ke dalam model, dihitung Jumlah Kuadrat Regresi untuk model regresi dengan memasukkan peubah Investment dan Unemployment Rate. Didapatkan JKR sebagai berikut : 

$ R(Investment|Imports, Exports, Industrial production growth rate) = 646029579496.3125 $
$ R(Unemployment rate|Imports, Exports, Industrial production growth rate) = 97814365746.3125 $

Pada model regresi yang memasukkan peubah Investment menghasilkan $s^2 =344404515287.2483$  dan diperoleh

$ f\ \ hitung=\ \frac{R(Investment|Imports, Exports, Industrial production growth rate)}{S^2} =\ 1.87578719447826  > {f\ tabel}_{0.95(1,111)}\ =3.926606812744345 $

karena $f_{hitung}$ lebih kecil dari $f_{tabel}$, makan peubah $Investment $ tidak masuk ke dalam model, begitu pula dengan peubah $Unemployment\ rate$. 



# Hasil Regresi

In [11]:
y_arr = np.array(data[Y])
x_arr = np.concatenate((b0,np.array(data[selected])),axis=1)

b_arr = b(x_arr,y_arr)
for i in range(len(selected)+1):
    print(f'b{i} = {b_arr[i][0]}')
    
print()

print('y = b0 + x1 * bi + x2 * b2 + ... + xk * bk')

print()

print(f'y({Y[0]}) = {b_arr[0][0]}',end='')
for i in range(len(selected)):
    print(f' + x({selected[i]}) * {b_arr[i+1][0]}',end='')
    
print()
print()
r2 = jkr(x_arr,y_arr)/JKT

print(f'R^2 = {r2} = {(r2*100):.2f}%')
print()

b0 = -145787.1198651685
b1 = 0.009513179892550409
b2 = -0.00324013691589963
b3 = 23775.676962988822

y = b0 + x1 * bi + x2 * b2 + ... + xk * bk

y(GDP) = -145787.1198651685 + x(Imports) * 0.009513179892550409 + x(Exports) * -0.00324013691589963 + x(Industrial production growth rate) * 23775.676962988822

R^2 = 0.8213010190028986 = 82.13%



Model persamaan regresi hanya mengandung Imports, Exports, dan Industrial Production Growth Rate. 
Persamaan taksiran model tersebut adalah:
$\beta_0 = -145787.1198651685$
$\beta_1 = 0.009513179892550409$
$\beta_2 = -0.00324013691589963$
$\beta_3 = 23775.676962988822$

$\widehat{y} = \beta_0 + x_1 * \beta_1 + x_2 * \beta_2 + x_3 * \beta_3$
$\widehat{y} = -145787.1198651685 + x_1 * 0.009513179892550409 + x_2 * -0.00324013691589963 + x_3 * 23775.676962988822$
$\widehat{y}_{GDP} = -145787.1198651685 + x_{Import} * 0.009513179892550409 + x_{Export}* -0.00324013691589963 + x_{IndustrialProductionGrowthRate} * 23775.676962988822$


Koefisien determinasi untuk model adalah :
$R^2 = 0.8213010190028986 \approx 82.13\% $

In [12]:
print('Karena pada praproses data terjadi penyederhanaan nilai export, import, dan gdp.')
print('maka:')
print()
print(f'y({Y[0]})/1000000 = {b_arr[0][0]} + (x({selected[0]})/1000) * {b_arr[1][0]} + (x({selected[1]})/1000) * {b_arr[2][0]} + x({selected[2]}) * {b_arr[3][0]}' )

print(f'y({Y[0]})= ({b_arr[0][0]} + (x({selected[0]})/1000) * {b_arr[1][0]} + (x({selected[1]})/1000) * {b_arr[2][0]} + x({selected[2]}) * {b_arr[3][0]}) * 1000000' )

print(f'y({Y[0]}) = {b_arr[0][0]*1000000} + x({selected[0]}) * {b_arr[1][0]*1000} + x({selected[1]}) * {b_arr[2][0]*1000} + x({selected[2]}) * {b_arr[3][0]*1000000}' )
print()
print('jadi nilai b yang dengan nilai x normal yaitu')
b_arr_copy = b_arr.copy()
b_arr_copy[0] = b_arr_copy[0]*1000000
b_arr_copy[1] = b_arr_copy[1]*1000
b_arr_copy[2] = b_arr_copy[2]*1000
b_arr_copy[3] = b_arr_copy[3]*1000000
for i in range(len(selected)+1):
    print(f'b{i} = {b_arr_copy[i][0]}')

Karena pada praproses data terjadi penyederhanaan nilai export, import, dan gdp.
maka:

y(GDP)/1000000 = -145787.1198651685 + (x(Imports)/1000) * 0.009513179892550409 + (x(Exports)/1000) * -0.00324013691589963 + x(Industrial production growth rate) * 23775.676962988822
y(GDP)= (-145787.1198651685 + (x(Imports)/1000) * 0.009513179892550409 + (x(Exports)/1000) * -0.00324013691589963 + x(Industrial production growth rate) * 23775.676962988822) * 1000000
y(GDP) = -145787119865.1685 + x(Imports) * 9.513179892550408 + x(Exports) * -3.2401369158996296 + x(Industrial production growth rate) * 23775676962.988823

jadi nilai b yang dengan nilai x normal yaitu
b0 = -145787119865.1685
b1 = 9.513179892550408
b2 = -3.2401369158996296
b3 = 23775676962.988823


Karena pada praproses data terjadi penyederhanaan nilai export, import, dan gdp.
maka:
$\frac{\widehat{y}_{GDP}}{1000000} = -145787.1198651685 + \frac{x_{import}}{1000}  * 0.009513179892550409 +\frac{x_{Export}}{1000}* -0.00324013691589963 + x_{IndustrialProductionGrowthRate} * 23775.676962988822$
$\widehat{y}_{GDP} = (-145787.1198651685 + \frac{x_{import}}{1000}  * 0.009513179892550409 +\frac{x_{Export}}{1000}* -0.00324013691589963 + x_{IndustrialProductionGrowthRate} * 23775.676962988822) * 1000000$
$\widehat{y}_{GDP} = -145787119865.1685 + x_{Import} * 9.513179892550408 + x_{Export}* -3.24013691589963 + x_{IndustrialProductionGrowthRate} * 23775676962.988823$


jadi nilai b yang dengan nilai x normal yaitu
$\beta_0 = -145787119865.1685$
$\beta_1 = 9.513179892550408$
$\beta_2 = -3.2401369158996296$
$\beta_3 = 23775676962.988823$

# Hasil Analisis

Peubah independent yang masuk ke dalam model regresi adalah Imports, Exports, dan Industrial Production Growth Rate. Hal ini menunjukkan bahwa perubahan dalam nilai exports, imports, dan tingkat pertumbuhan produksi industri akan memiliki dampak yang signifikan pada perubahan nilai GDP. Oleh karena itu, ketiga variabel tersebut dianggap penting dan dimasukkan ke dalam model regresi untuk memprediksi GDP.

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=6e4815cf-a2e9-405c-bd55-ed8031ee0253' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>